---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [ ]:
import pandas as pd
import numpy as np
import re
doc = []
with open('./../Datasets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(5)

In [ ]:
def date_sorter():
        # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    # Feb 2009; Sep 2009; Oct 2010
    # 6/2008; 12/2009
    # 2009; 2010
    df2 = df.str.extractall(r'(?P<Dates>(?:(?:\d{1,2})[-/](?:\d{1,2}[-/])(?:\d{2,4})|(?:\d{1,2})?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,-. ]{0,2}(?:\d{0,2}(?:nd|th|st)?)?[,-. ]{0,2}\d{2,4}|\d{4}))')
    df2["Dates"] = df2["Dates"].str.replace("[-., ]+","/",regex=True)
    df2 = df2.apply(add_proper_dates, axis =1)
    df2["Dates_new"].replace({"Jan[a-z]*":"01","Feb[a-z]*":"02","Mar[a-z]*":"03","Apr[a-z]*":"04","May[a-z]*":"05","Jun[a-z]*":"06","Jul[a-z]*":"07","Aug[a-z]*":"08","Sep[a-z]*":"09","Oct[a-z]*":"10","Nov[a-z]*":"11","Dec[a-z]*":"12"},inplace = True, regex=True)
    df2 = df2[df2["Dates_new"].isnull()!=True]

    df2 = (df2.reset_index()
              .drop(["level_0","match","Dates"], axis=1)
              .rename({"Dates_new":"Dates"}, axis=1))

    df2["Dates"] = pd.to_datetime(df2["Dates"])

    result = (df2.sort_values("Dates",ascending=True)
                  .reset_index()
                  .rename({"index":"position"},axis =1)
                  .iloc[:,0])
    # Your code here
    
    return pd.Series(result.tolist())

In [ ]:
date_sorter()

In [ ]:
# def test():
#     import pandas as pd

#     fun = date_sorter()
#     res = 'Data Type Test (Series?): '
#     res += ['Failed\n','Passed\n'][type(fun)==pd.Series]
    
#     res += 'Data Shape Test ((500,)?): '
#     res += ['Failed\n','Passed\n'][fun.shape==(500,)]
    
#     res += 'Index Values Test (range(500)?): '
#     res += ['Failed\n','Passed\n'][fun.index.tolist()==list(range(500))]
    
#     res += 'Values Test (0-499): '
#     res += ['Failed\n','Passed\n'][all((fun<500) & (fun>=0))]
    
#     return res
# print(test())